In [28]:
import pandas as pd

In [29]:
predictions = pd.read_csv(f"../data/predictions/predictions-{11}-{19}.csv")
final_predictions = pd.read_csv(f"../data/predictions/predictions-final-{11}-{17}.csv")

In [30]:
features = pd.read_csv("../data/tsfeatures/tsfeatures.csv")\
    .dropna(axis = 1, how = 'all')

features = features.rename({"unique_id":"store_item"}, axis=1)

In [31]:
final_feats = pd.read_csv(f"../data/tsfeatures/tsfeatures_final.csv")\
    .dropna(axis = 1, how = 'all')
    
final_feats = final_feats.rename({"unique_id":"store_item"}, axis=1)

In [32]:
from sklearn.impute import KNNImputer

def impute_hurst(df):
    
    imputer = KNNImputer()
    knn_impute = df.select_dtypes(include=['float64'])
    knn_imputed = imputer.fit_transform(knn_impute)
    knn_imputed = pd.DataFrame(knn_imputed, columns=knn_impute.columns)
    return knn_imputed[["hurst", "entropy"]]

features[["hurst", "entropy"]] = impute_hurst(features)
final_feats[["hurst", "entropy"]] = impute_hurst(final_feats)

In [33]:
def drop_same_val_colums(df):
    nunique = df.nunique()
    cols_to_drop = nunique[nunique == 1].index
    return df.drop(cols_to_drop, axis=1)

features = drop_same_val_colums(features)
final_feats = drop_same_val_colums(final_feats)

In [34]:
predictions = predictions.rename({'Unnamed: 0':"date"}, axis=1)
predictions.date = pd.to_datetime(predictions.date)

final_predictions = final_predictions.rename({'Unnamed: 0':"date"}, axis=1)
final_predictions.date = pd.to_datetime(final_predictions.date)

In [61]:
def smape(F):
    return 100/len(predictions.sales) * np.sum(2 * np.abs(F - predictions.sales) / 
                                               (np.abs(predictions.sales) + np.abs(F)))

predictions
smape(predictions.autoreg)

predictions.iloc[:, 3:].apply(smape).sort_values().round(2)

prophet           12.89
ardl              16.83
neural_prophet    18.44
exp_smooth        19.31
autoreg           23.93
xgb_preds         30.35
dtype: float64

In [37]:

from epftoolbox.evaluation import MASE
MASE(p_pred = predictions.sales,
     p_real= predictions.autoreg, 
     p_real_in=predictions.sales)




KeyError: "None of [DatetimeIndex(['1969-12-31 00:00:00.000000168',\n               '1969-12-31 00:00:00.000000169',\n               '1969-12-31 00:00:00.000000170',\n               '1969-12-31 00:00:00.000000171',\n               '1969-12-31 00:00:00.000000172',\n               '1969-12-31 00:00:00.000000173',\n               '1969-12-31 00:00:00.000000174',\n               '1969-12-31 00:00:00.000000175',\n               '1969-12-31 00:00:00.000000176',\n               '1969-12-31 00:00:00.000000177',\n               ...\n               '1969-12-31 00:00:00.000273490',\n               '1969-12-31 00:00:00.000273491',\n               '1969-12-31 00:00:00.000273492',\n               '1969-12-31 00:00:00.000273493',\n               '1969-12-31 00:00:00.000273494',\n               '1969-12-31 00:00:00.000273495',\n               '1969-12-31 00:00:00.000273496',\n               '1969-12-31 00:00:00.000273497',\n               '1969-12-31 00:00:00.000273498',\n               '1969-12-31 00

In [ ]:
predictions.to_csv("../data/predictions/errors.csv", index=False)

In [ ]:
error = predictions.iloc[:, 3:9] # fcast_models
predictions.drop(predictions.iloc[:, 3:9], inplace=True, axis=1)
errors = error.sub(predictions.sales, axis=0)

In [ ]:
from helper import create_features

In [ ]:
final_predictions = pd.concat([final_predictions, create_features(final_predictions)], axis=1)
predictions = pd.concat([predictions, create_features(predictions)], axis=1)

In [ ]:
final_df = pd.merge(predictions, features, right_on="store_item", left_on= "store_item")
y = errors.abs().idxmin(axis=1)
X = final_df.drop(final_df.columns[[0, 1, 2, 19]], inplace=False, axis=1)
X = X.loc[:,~X.columns.duplicated()]

In [ ]:
y.to_csv("../data/forecasts_for_ml/y.csv", index=False)
X.to_csv("../data/forecasts_for_ml/X.csv", index=False)

In [ ]:
final_predictions_df = pd.merge(final_predictions, final_feats, right_on="store_item", left_on= "store_item")
final_predictions_df = final_predictions_df[X.columns]
final_predictions_df = final_predictions_df.loc[:,~final_predictions_df.columns.duplicated()]
final_predictions_df.to_csv("../data/forecasts_for_ml/X_final.csv", index=False)